<a href="https://colab.research.google.com/github/kd365/faafall22/blob/main/cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
 
 
uploaded = files.upload()

Saving ntsb_post_2008.csv to ntsb_post_2008.csv


In [3]:
import pandas as pd


pd.set_option('display.max_rows', 150)

ntsb_raw = pd.read_csv("ntsb_post_2008.csv", encoding='latin-1', low_memory=False)

print(ntsb_raw.dtypes)

ï»¿aircraft.ev_id            object
far_part                     object
flight_plan_activated        object
damage                       object
acft_make                    object
acft_model                   object
cert_max_gr_wt              float64
num_eng                     float64
type_last_insp               object
date_last_insp               object
afm_hrs_last_insp           float64
afm_hrs                     float64
type_fly                     object
dprt_apt_id                  object
dprt_city                    object
dprt_state                   object
dprt_country                 object
dprt_time                   float64
dprt_timezn                 float64
dest_same_local             float64
dest_apt_id                  object
dest_city                    object
dest_state                   object
dest_country                 object
phase_flt_spec              float64
afm_hrs_since                object
rwy_num                      object
rwy_len                     

In [5]:
#Drop dprt_timezn

ntsb_raw.head()

,ï»¿aircraft.ev_id,far_part,flight_plan_activated,damage,acft_make,acft_model,cert_max_gr_wt,num_eng,type_last_insp,date_last_insp,...,Flight_Crew_Aircraft_Key,crew_no,crew_category,crew_age,crew_sex,crew_city,crew_res_state,med_certf,pilot_flying,crew_inj_level
0,20160824X03156,NaN,NaN,NaN,CESSNA,550,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20160831X40558,NUSN,NaN,DEST,ROBINSON,R66,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20181029X90852,UNK,NaN,SUBS,PIPER,L4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20160830X31131,NUSC,NaN,DEST,ROBINSON,R44,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20181031X51508,NUSC,NaN,SUBS,AIRBUS,A330,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
ntsb_rv1 = ntsb_raw.dropna(axis=1, how='all')
print(ntsb_rv1.dtypes)

ï»¿aircraft.ev_id            object
far_part                     object
flight_plan_activated        object
damage                       object
acft_make                    object
acft_model                   object
cert_max_gr_wt              float64
num_eng                     float64
type_last_insp               object
date_last_insp               object
afm_hrs_last_insp           float64
afm_hrs                     float64
type_fly                     object
dprt_apt_id                  object
dprt_city                    object
dprt_state                   object
dprt_country                 object
dprt_time                   float64
dest_apt_id                  object
dest_city                    object
dest_state                   object
dest_country                 object
afm_hrs_since                object
rwy_num                      object
rwy_len                     float64
rwy_width                   float64
fuel_on_board               float64
ifr_equipped_cert           

In [7]:
ntsb_rv1.head()

,ï»¿aircraft.ev_id,far_part,flight_plan_activated,damage,acft_make,acft_model,cert_max_gr_wt,num_eng,type_last_insp,date_last_insp,...,Flight_Crew_Aircraft_Key,crew_no,crew_category,crew_age,crew_sex,crew_city,crew_res_state,med_certf,pilot_flying,crew_inj_level
0,20160824X03156,NaN,NaN,NaN,CESSNA,550,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20160831X40558,NUSN,NaN,DEST,ROBINSON,R66,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20181029X90852,UNK,NaN,SUBS,PIPER,L4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20160830X31131,NUSC,NaN,DEST,ROBINSON,R44,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20181031X51508,NUSC,NaN,SUBS,AIRBUS,A330,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(ntsb_rv1[['wx_obs_time']].to_string(index=False)) 

 wx_obs_time
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
      1335.0
      1500.0
         NaN
         NaN
         NaN
      1755.0
         NaN
         NaN
         NaN
      2053.0
         NaN
         NaN
      1935.0
         NaN
         NaN
         NaN
         NaN
      2155.0
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
      1547.0
         NaN
         NaN
      1453.0
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
         NaN
      1553.0
         NaN
         NaN
      2200.0
         NaN

In [9]:
pd.options.mode.chained_assignment = None  # default='warn'

In [10]:
ntsb_rv1['date_last_insp'] =  pd.to_datetime(ntsb_rv1['date_last_insp'], infer_datetime_format=True)


In [11]:
#ntsb_rv2= ntsb_rv1.drop(['Aircraft_Key'], axis=1)

KeyError: ignored

In [13]:
print(ntsb_rv1.dtypes)

ï»¿aircraft.ev_id                   object
far_part                            object
flight_plan_activated               object
damage                              object
acft_make                           object
acft_model                          object
cert_max_gr_wt                     float64
num_eng                            float64
type_last_insp                      object
date_last_insp              datetime64[ns]
afm_hrs_last_insp                  float64
afm_hrs                            float64
type_fly                            object
dprt_apt_id                         object
dprt_city                           object
dprt_state                          object
dprt_country                        object
dprt_time                          float64
dest_apt_id                         object
dest_city                           object
dest_state                          object
dest_country                        object
afm_hrs_since                       object
rwy_num    

In [15]:
ntsb_rv1['ev_date'] =  pd.to_datetime(ntsb_rv1['ev_date'], infer_datetime_format=True)

In [16]:
ntsb_rv1['wx_obs_time'] =  pd.to_datetime(ntsb_rv1['wx_obs_time'], format = "%H%M")

ValueError: ignored

In [ ]:
#Walkthrough at https://inria.github.io/scikit-learn-mooc/python_scripts/03_categorical_pipeline_column_transformer.html
#preprocessing different for categorical vs numerical columns

from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(ntsb_raw)
categorical_columns = categorical_columns_selector(ntsb_raw)